## 전국 신규 민간 아파트 분양 가격 동향 자료 전처리 및 분석

1. 데이터 : 공공데이터 포탈에서 제공, 2015년 10월부터 2018년 7월까지 주택분양 가격 정보 제공
- 참고 : 파일명 수정된 상태(한글 파일 -> 영문과 숫자)
* 한국 데이터셋 파일들 : 인코딩 수정 또는 확인 필수(cp949, euc-kr  => utf-8로 변환해야 한다.)
- 단순 경고 발생 (skip 명령어 적용)



In [1]:
# 단순 경고메시지는 무시해주는 기능
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
import re

2015년 1월 ~ 2018년 7월까지의 신규 아파트 분양정보 확인 및 전처리(평당 분양가격 추가)

In [3]:
# avg18.07.csv 임포트
pre_sale = pd.read_csv('avg18.07.csv', encoding = 'utf-8')
pre_sale

,지역명,규모구분,연도,월,분양가격(㎡)
0,서울,전체,2015,10,5841
1,서울,전용면적 60㎡이하,2015,10,5652
2,서울,전용면적 60㎡초과 85㎡이하,2015,10,5882
3,서울,전용면적 85㎡초과 102㎡이하,2015,10,5721
4,서울,전용면적 102㎡초과,2015,10,5879
5,인천,전체,2015,10,3163
6,인천,전용면적 60㎡이하,2015,10,3488
7,인천,전용면적 60㎡초과 85㎡이하,2015,10,3119
8,인천,전용면적 85㎡초과 102㎡이하,2015,10,3545
9,인천,전용면적 102㎡초과,2015,10,3408


In [4]:
'''
1. 5개 머리부분만
2. 5개 꼬리부분만
3. 결측치 있는거 찾아내기
4. shape랑 row / column수 확인
'''

'\n1. 5개 머리부분만\n2. 5개 꼬리부분만\n3. 결측치 있는거 찾아내기\n4. shape랑 row / column수 확인\n'

In [137]:
pre_sale.head()

,지역명,규모구분,연도,월,분양가격(㎡),분양가격,평당분양가격
0,서울,전체,2015,10,5841,"5,841","19,275"
1,서울,전용면적 60㎡이하,2015,10,5652,"5,652","18,652"
2,서울,전용면적 60㎡초과 85㎡이하,2015,10,5882,"5,882","19,411"
3,서울,전용면적 85㎡초과 102㎡이하,2015,10,5721,"5,721","18,879"
4,서울,전용면적 102㎡초과,2015,10,5879,"5,879","19,401"


In [138]:
pre_sale.tail()

,지역명,규모구분,연도,월,분양가격(㎡),분양가격,평당분양가격
2885,제주,전체,2018,7,3334,"3,334","11,002"
2886,제주,전용면적 60㎡이하,2018,7,NaN,nan,nan
2887,제주,전용면적 60㎡초과 85㎡이하,2018,7,3356,"3,356","11,075"
2888,제주,전용면적 85㎡초과 102㎡이하,2018,7,3226,"3,226","10,646"
2889,제주,전용면적 102㎡초과,2018,7,2990,"2,990","9,867"


In [139]:
# 특정 칼럼의 null 갯수 합
'''
nans = lambda df: df[df.isnull().any(axis=1)]
nans(pre_sale)
'''
pre_sale["분양가격(㎡)"].isna().sum()

140

In [140]:
pre_sale.shape

(2890, 7)

In [141]:
# 칼럼별 타입 확인
pre_sale.dtypes

지역명         object
규모구분        object
연도           int64
월            int64
분양가격(㎡)     object
분양가격       float64
평당분양가격     float64
dtype: object

In [142]:
# null 갯수를 제외한 유효한 데이터 갯수 확인
pre_sale.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2890 entries, 0 to 2889
Data columns (total 7 columns):
지역명        2890 non-null object
규모구분       2890 non-null object
연도         2890 non-null int64
월          2890 non-null int64
분양가격(㎡)    2750 non-null object
분양가격       2658 non-null float64
평당분양가격     2658 non-null float64
dtypes: float64(2), int64(2), object(3)
memory usage: 158.1+ KB


In [143]:
# 분양가격 값만 검색해 보기
pre_sale['분양가격(㎡)'] # 옵젝타입이라 수치연산 불가능

0       5841
1       5652
2       5882
3       5721
4       5879
5       3163
6       3488
7       3119
8       3545
9       3408
10      3138
11      3126
12      3239
13      3496
14      3680
15      3112
16      2950
17      2999
18      2957
19      3500
20      2682
21      2614
22      2696
23      2557
24      2598
25      2420
26      2253
27      2439
28          
29          
        ... 
2860    2467
2861    2452
2862    2467
2863    2657
2864    2580
2865    2333
2866    2655
2867    2306
2868    2636
2869    2850
2870    2357
2871    2228
2872    2348
2873    2627
2874    2469
2875    2674
2876    2471
2877    2654
2878     NaN
2879    2903
2880    2696
2881    2748
2882    2673
2883    2857
2884    2839
2885    3334
2886     NaN
2887    3356
2888    3226
2889    2990
Name: 분양가격(㎡), Length: 2890, dtype: object

In [144]:
# 분양가격으로 새로운 칼럼을 도출(연산 수행 가능한 타입으로 변환)
# raw 데이터는 유지 + 새로운 칼럼 추가 (분양가격 / 평당 분양가격)

# 존재하는 칼럼으로 새로운 Series객체 생성
pre_sale_price = pre_sale['분양가격(㎡)']
pre_sale_price
# .copy() 쓰면 안되나??

0       5841
1       5652
2       5882
3       5721
4       5879
5       3163
6       3488
7       3119
8       3545
9       3408
10      3138
11      3126
12      3239
13      3496
14      3680
15      3112
16      2950
17      2999
18      2957
19      3500
20      2682
21      2614
22      2696
23      2557
24      2598
25      2420
26      2253
27      2439
28          
29          
        ... 
2860    2467
2861    2452
2862    2467
2863    2657
2864    2580
2865    2333
2866    2655
2867    2306
2868    2636
2869    2850
2870    2357
2871    2228
2872    2348
2873    2627
2874    2469
2875    2674
2876    2471
2877    2654
2878     NaN
2879    2903
2880    2696
2881    2748
2882    2673
2883    2857
2884    2839
2885    3334
2886     NaN
2887    3356
2888    3226
2889    2990
Name: 분양가격(㎡), Length: 2890, dtype: object

In [145]:
'''
errors = 'coerce' : 에러발생 시 무시
errors = 'raise' : 에러발생 시 표현
'''

pre_sale['분양가격'] = pd.to_numeric(pre_sale_price, errors = 'coerce')

In [146]:
pre_sale['분양가격'].dtype

dtype('float64')

In [147]:
pre_sale

,지역명,규모구분,연도,월,분양가격(㎡),분양가격,평당분양가격
0,서울,전체,2015,10,5841,"5,841","19,275"
1,서울,전용면적 60㎡이하,2015,10,5652,"5,652","18,652"
2,서울,전용면적 60㎡초과 85㎡이하,2015,10,5882,"5,882","19,411"
3,서울,전용면적 85㎡초과 102㎡이하,2015,10,5721,"5,721","18,879"
4,서울,전용면적 102㎡초과,2015,10,5879,"5,879","19,401"
5,인천,전체,2015,10,3163,"3,163","10,438"
6,인천,전용면적 60㎡이하,2015,10,3488,"3,488","11,510"
7,인천,전용면적 60㎡초과 85㎡이하,2015,10,3119,"3,119","10,293"
8,인천,전용면적 85㎡초과 102㎡이하,2015,10,3545,"3,545","11,698"
9,인천,전용면적 102㎡초과,2015,10,3408,"3,408","11,246"


In [148]:
# 평단분양가격 이라는 새로운 칼럼 생성해서
# 분양가격 * 3.3 연산

In [149]:
pre_sale['평당분양가격'] = pre_sale['분양가격'] * 3.3

In [150]:
pre_sale['평당분양가격'].round()

0      19,275
1      18,652
2      19,411
3      18,879
4      19,401
5      10,438
6      11,510
7      10,293
8      11,698
9      11,246
10     10,355
11     10,316
12     10,689
13     11,537
14     12,144
15     10,270
16      9,735
17      9,897
18      9,758
19     11,550
20      8,851
21      8,626
22      8,897
23      8,438
24      8,573
25      7,986
26      7,435
27      8,049
28        nan
29        nan
        ...  
2860    8,141
2861    8,092
2862    8,141
2863    8,768
2864    8,514
2865    7,699
2866    8,762
2867    7,610
2868    8,699
2869    9,405
2870    7,778
2871    7,352
2872    7,748
2873    8,669
2874    8,148
2875    8,824
2876    8,154
2877    8,758
2878      nan
2879    9,580
2880    8,897
2881    9,068
2882    8,821
2883    9,428
2884    9,369
2885   11,002
2886      nan
2887   11,075
2888   10,646
2889    9,867
Name: 평당분양가격, Length: 2890, dtype: float64

In [151]:
pre_sale.iloc[28]

지역명                       광주
규모구분       전용면적 85㎡초과 102㎡이하
연도                      2015
월                         10
분양가격(㎡)                     
분양가격                     NaN
평당분양가격                   NaN
Name: 28, dtype: object

In [152]:
# 결측치를 단순 메모장에서 필수 확인
# 분양가격, 평당분양가격 컬럼은 공백도 null로 집계
pre_sale['분양가격(㎡)'].isnull().sum()

140

In [153]:
pre_sale['분양가격'].isnull().sum()

232

In [154]:
pre_sale['평당분양가격'].isnull().sum()

232

In [155]:
pre_sale.dtypes

지역명         object
규모구분        object
연도           int64
월            int64
분양가격(㎡)     object
분양가격       float64
평당분양가격     float64
dtype: object

In [156]:
pre_sale.describe()

,연도,월,분양가격,평당분양가격
count,"2,890","2,890","2,658","2,658"
mean,"2,017",6,"3,063","10,109"
std,1,4,"1,113","3,672"
min,"2,015",1,"1,868","6,164"
25%,"2,016",3,"2,348","7,748"
50%,"2,017",6,"2,743","9,052"
75%,"2,017",10,"3,294","10,869"
max,"2,018",12,"8,098","26,723"


In [25]:
pre_sale.describe(include=[np.object])

,지역명,규모구분,분양가격(㎡)
count,2890,2890,2750
unique,17,5,1305
top,광주,전용면적 60㎡이하,2221
freq,170,578,17


In [26]:
#연도 기준으로 2017년 데이터 검색 

In [28]:
#지역명 확인
# select distinct 지역명 from pre_sale
pre_sale['지역명'].value_counts()

광주    170
강원    170
충남    170
부산    170
세종    170
서울    170
충북    170
제주    170
경기    170
인천    170
울산    170
경북    170
전남    170
대전    170
전북    170
대구    170
경남    170
Name: 지역명, dtype: int64

### 전국 평균 분양 가격

In [32]:
# 연도별 수치 타입인 분양가격과 평균분양가격 집계정보 확인
pre_sale.groupby(pre_sale.연도).describe().T

연도                    2015          2016          2017          2018
분양가격   count    243.000000    984.000000    899.000000    532.000000
       mean    2788.707819   2934.250000   3139.541713   3298.567669
       std      976.861182   1071.441561   1107.584870   1195.955014
       min     1868.000000   1900.000000   1976.000000   2076.000000
       25%     2225.000000   2282.000000   2365.000000   2471.000000
       50%     2516.000000   2672.000000   2849.000000   2914.000000
       75%     3025.500000   3148.500000   3456.000000   3647.250000
       max     7092.000000   8096.000000   7887.000000   8098.000000
월      count    255.000000   1020.000000   1020.000000    595.000000
       mean      11.000000      6.500000      6.500000      4.000000
       std        0.818102      3.453746      3.453746      2.001683
       min       10.000000      1.000000      1.000000      1.000000
       25%       10.000000      3.750000      3.750000      2.000000
       50%       11.000000      6.500000      6.500000      4.000000
       75%       12.000000      9.250000      9.250000      6.000000
       max       12.000000     12.000000     12.000000      7.000000
평당분양가격 count    243.000000    984.000000    899.000000    532.000000
       mean    9202.735802   9683.025000  10360.487653  10885.273308
       std     3223.641902   3535.757150   3655.030071   3946.651548
       min     6164.400000   6270.000000   6520.800000   6850.800000
       25%     7342.500000   7530.600000   7804.500000   8154.300000
       50%     8302.800000   8817.600000   9401.700000   9616.200000
       75%     9984.150000  10390.050000  11404.800000  12035.925000
       max    23403.600000  26716.800000  26027.100000  26723.400000

In [33]:
pre_sale.pivot_table?

In [34]:
#pre_sale.pivot_table(values=None, index=None, columns=None)

In [ ]:
'''
피벗 함수 이해를 위한 전처리
'''

In [157]:
pre_sale.dtypes

지역명         object
규모구분        object
연도           int64
월            int64
분양가격(㎡)     object
분양가격       float64
평당분양가격     float64
dtype: object

In [158]:
pre_sale['규모구분']

0                      전체
1              전용면적 60㎡이하
2        전용면적 60㎡초과 85㎡이하
3       전용면적 85㎡초과 102㎡이하
4             전용면적 102㎡초과
5                      전체
6              전용면적 60㎡이하
7        전용면적 60㎡초과 85㎡이하
8       전용면적 85㎡초과 102㎡이하
9             전용면적 102㎡초과
10                     전체
11             전용면적 60㎡이하
12       전용면적 60㎡초과 85㎡이하
13      전용면적 85㎡초과 102㎡이하
14            전용면적 102㎡초과
15                     전체
16             전용면적 60㎡이하
17       전용면적 60㎡초과 85㎡이하
18      전용면적 85㎡초과 102㎡이하
19            전용면적 102㎡초과
20                     전체
21             전용면적 60㎡이하
22       전용면적 60㎡초과 85㎡이하
23      전용면적 85㎡초과 102㎡이하
24            전용면적 102㎡초과
25                     전체
26             전용면적 60㎡이하
27       전용면적 60㎡초과 85㎡이하
28      전용면적 85㎡초과 102㎡이하
29            전용면적 102㎡초과
              ...        
2860                   전체
2861           전용면적 60㎡이하
2862     전용면적 60㎡초과 85㎡이하
2863    전용면적 85㎡초과 102㎡이하
2864          전용면적 102㎡초과
2865                   전체
2866           전용면적 60㎡이하
2867     전용면

In [37]:
# 전용면적 60㎡이하 인 데이터만 검색
a=pre_sale.loc[pre_sale['규모구분']=='전용면적 60㎡이하']

In [38]:
a.shape

(578, 7)

In [42]:
b=a.loc[a['연도']==2015]
b.shape

(51, 7)

In [43]:
b['평당분양가격'].mean()

8951.525000000001

In [44]:
# "평당분양가격"을 "규모구분"과 "연도"별로 평균값 구하기

In [159]:
pre_sale.pivot_table("평당분양가격","규모구분","연도")

연도,2015,2016,2017,2018
규모구분,,,,
전용면적 102㎡초과,"9,837","10,389","11,334","11,235"
전용면적 60㎡이하,"8,952","9,399","10,246","10,912"
전용면적 60㎡초과 85㎡이하,"8,892","9,296","9,825","10,464"
전용면적 85㎡초과 102㎡이하,"9,519","10,122","10,540","11,483"
전체,"8,893","9,293","9,901","10,563"


In [160]:
yearUp=all.pre_sale.pivot_table("평당분양가격","규모구분","연도")
yearUp

AttributeError: 'builtin_function_or_method' object has no attribute 'pre_sale'

In [55]:
# 규모 구분이 전체로 되어있는 금액으로 연도별 변동금액 알아보기

In [57]:
# 평당분양가격, 지역명, 연도 관련해 pivot_table()

In [58]:
yearUp['변동액']=yearUp[2018]-yearUp[2015]
yearUp

NameError: name 'yearUp' is not defined

In [62]:
# 소수 데이터들을 정수화 시킬 수 있는 설정 + 천단위 쉼표 처리 추가
pd.options.display.float_format='{:,.0f}'.format
yearUp

NameError: name 'yearUp' is not defined

In [63]:
'''
? 지역별 평당 분양가격 합계 구ㅏ기
피벗의 인수값 : 평당분양가격, 규모구분, ?
?
'''


'\n? 지역별 평당 분양가격 합계 구ㅏ기\n피벗의 인수값 : 평당분양가격, 규모구분, ?\n?\n'

In [170]:
# avg15.09.csv로 DataFrame 객체 생성하기
df = pd.read_csv('avg15.09.csv', header = 0, encoding = 'utf-8', skiprows=1)
df

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26
0,구분,NaN,2013년,2014년,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,(단위: 천원/3.3㎡)
1,시도,시군구,12월,1월,2월,3월,4월,5월,6월,7월,...,3월,4월,5월,6월,7월,8월,9월,전월비,전년말비,전년동월비
2,전국,NaN,"8,059","8,130","8,195","8,204","8,289","8,358","8,344","8,333",...,"8,563","8,613","8,624","8,627","8,643","8,678","8,665",-13,82,207
3,서울,NaN,"18,189","17,925","17,925","18,016","18,098","19,446","18,867","18,742",...,"19,415","18,842","18,367","18,374","18,152","18,443","17,969",-474,"-2,300","-1,434"
4,6대광역시,부산,"8,111","8,111","9,078","8,965","9,402","9,501","9,453","9,457",...,"9,279","9,327","9,345","9,515","9,559","9,581","9,608",1,430,477
5,NaN,대구,"8,080","8,080","8,077","8,101","8,267","8,274","8,360","8,360",...,"8,441","8,446","8,568","8,542","8,542","8,795","8,863",27,400,350
6,NaN,인천,"10,204","10,204","10,408","10,408","10,000","9,844","10,058","9,974",...,"9,876","9,938","10,551","10,443","10,443","10,449","10,450",-162,-150,-131
7,NaN,광주,"6,098","7,326","7,611","7,346","7,346","7,523","7,659","7,612",...,"7,861","7,914","7,877","7,881","8,089","8,231","8,083",-148,334,281
8,NaN,대전,"8,321","8,321","8,321","8,341","8,341","8,341","8,333","8,333",...,"8,067","8,145","8,272","8,079","8,079","8,079","7,917",68,610,414
9,NaN,울산,"8,090","8,090","8,090","8,153","8,153","8,153","8,153","8,153",...,"8,629","9,380","9,192","9,190","9,190","9,215","9,215",0,324,722


In [171]:
year = df.iloc[0]
year

Unnamed: 0                구분
Unnamed: 1               NaN
Unnamed: 2             2013년
Unnamed: 3             2014년
Unnamed: 4               NaN
Unnamed: 5               NaN
Unnamed: 6               NaN
Unnamed: 7               NaN
Unnamed: 8               NaN
Unnamed: 9               NaN
Unnamed: 10              NaN
Unnamed: 11              NaN
Unnamed: 12              NaN
Unnamed: 13              NaN
Unnamed: 14              NaN
Unnamed: 15            2015년
Unnamed: 16              NaN
Unnamed: 17              NaN
Unnamed: 18              NaN
Unnamed: 19              NaN
Unnamed: 20              NaN
Unnamed: 21              NaN
Unnamed: 22              NaN
Unnamed: 23              NaN
Unnamed: 24              NaN
Unnamed: 25              NaN
Unnamed: 26    (단위: 천원/3.3㎡)
Name: 0, dtype: object

In [172]:
month = df.iloc[1]
month

Unnamed: 0        시도
Unnamed: 1       시군구
Unnamed: 2       12월
Unnamed: 3        1월
Unnamed: 4        2월
Unnamed: 5        3월
Unnamed: 6        4월
Unnamed: 7        5월
Unnamed: 8        6월
Unnamed: 9        7월
Unnamed: 10       8월
Unnamed: 11       9월
Unnamed: 12      10월
Unnamed: 13      11월
Unnamed: 14      12월
Unnamed: 15       1월
Unnamed: 16       2월
Unnamed: 17       3월
Unnamed: 18       4월
Unnamed: 19       5월
Unnamed: 20       6월
Unnamed: 21       7월
Unnamed: 22       8월
Unnamed: 23       9월
Unnamed: 24      전월비
Unnamed: 25     전년말비
Unnamed: 26    전년동월비
Name: 1, dtype: object

In [173]:
'''
python의 enumerate() 함수의 메뉴얼 참조 필수

year와 month의 데이터를 결합해서 "2014년 4월.." 형태로 가공 필수

1. join()/year[index]/month[index]
2. 2014이후부터 2015년 이전까지 2014로 채우기
3. 2015이후부터는 2015로 채우기
'''
for i, y in enumerate(year):
    if i > 2 and i < 15:
        year[i] = ' '.join(['2014년', month[i]])
    elif i >= 15:
        year[i] = ' '.join(['2015년', month[i]])
    elif i == 2:
        year[i] = ' '.join([year[i]])
    elif i ==1:
        year[i] = '시군구'
year

Unnamed: 0              구분
Unnamed: 1             시군구
Unnamed: 2           2013년
Unnamed: 3        2014년 1월
Unnamed: 4        2014년 2월
Unnamed: 5        2014년 3월
Unnamed: 6        2014년 4월
Unnamed: 7        2014년 5월
Unnamed: 8        2014년 6월
Unnamed: 9        2014년 7월
Unnamed: 10       2014년 8월
Unnamed: 11       2014년 9월
Unnamed: 12      2014년 10월
Unnamed: 13      2014년 11월
Unnamed: 14      2014년 12월
Unnamed: 15       2015년 1월
Unnamed: 16       2015년 2월
Unnamed: 17       2015년 3월
Unnamed: 18       2015년 4월
Unnamed: 19       2015년 5월
Unnamed: 20       2015년 6월
Unnamed: 21       2015년 7월
Unnamed: 22       2015년 8월
Unnamed: 23       2015년 9월
Unnamed: 24      2015년 전월비
Unnamed: 25     2015년 전년말비
Unnamed: 26    2015년 전년동월비
Name: 0, dtype: object

In [174]:
df2=df.copy()

In [175]:
df2.columns=year
df2

,구분,시군구,2013년,2014년 1월,2014년 2월,2014년 3월,2014년 4월,2014년 5월,2014년 6월,2014년 7월,...,2015년 3월,2015년 4월,2015년 5월,2015년 6월,2015년 7월,2015년 8월,2015년 9월,2015년 전월비,2015년 전년말비,2015년 전년동월비
0,구분,시군구,2013년,2014년 1월,2014년 2월,2014년 3월,2014년 4월,2014년 5월,2014년 6월,2014년 7월,...,2015년 3월,2015년 4월,2015년 5월,2015년 6월,2015년 7월,2015년 8월,2015년 9월,2015년 전월비,2015년 전년말비,2015년 전년동월비
1,시도,시군구,12월,1월,2월,3월,4월,5월,6월,7월,...,3월,4월,5월,6월,7월,8월,9월,전월비,전년말비,전년동월비
2,전국,NaN,"8,059","8,130","8,195","8,204","8,289","8,358","8,344","8,333",...,"8,563","8,613","8,624","8,627","8,643","8,678","8,665",-13,82,207
3,서울,NaN,"18,189","17,925","17,925","18,016","18,098","19,446","18,867","18,742",...,"19,415","18,842","18,367","18,374","18,152","18,443","17,969",-474,"-2,300","-1,434"
4,6대광역시,부산,"8,111","8,111","9,078","8,965","9,402","9,501","9,453","9,457",...,"9,279","9,327","9,345","9,515","9,559","9,581","9,608",1,430,477
5,NaN,대구,"8,080","8,080","8,077","8,101","8,267","8,274","8,360","8,360",...,"8,441","8,446","8,568","8,542","8,542","8,795","8,863",27,400,350
6,NaN,인천,"10,204","10,204","10,408","10,408","10,000","9,844","10,058","9,974",...,"9,876","9,938","10,551","10,443","10,443","10,449","10,450",-162,-150,-131
7,NaN,광주,"6,098","7,326","7,611","7,346","7,346","7,523","7,659","7,612",...,"7,861","7,914","7,877","7,881","8,089","8,231","8,083",-148,334,281
8,NaN,대전,"8,321","8,321","8,321","8,341","8,341","8,341","8,333","8,333",...,"8,067","8,145","8,272","8,079","8,079","8,079","7,917",68,610,414
9,NaN,울산,"8,090","8,090","8,090","8,153","8,153","8,153","8,153","8,153",...,"8,629","9,380","9,192","9,190","9,190","9,215","9,215",0,324,722


In [186]:
df2.drop([0,1],inplace=True)
df2

,구분,시군구,2013년,2014년 1월,2014년 2월,2014년 3월,2014년 4월,2014년 5월,2014년 6월,2014년 7월,...,2015년 3월,2015년 4월,2015년 5월,2015년 6월,2015년 7월,2015년 8월,2015년 9월,2015년 전월비,2015년 전년말비,2015년 전년동월비
2,전국,NaN,"8,059","8,130","8,195","8,204","8,289","8,358","8,344","8,333",...,"8,563","8,613","8,624","8,627","8,643","8,678","8,665",-13,82,207
3,서울,NaN,"18,189","17,925","17,925","18,016","18,098","19,446","18,867","18,742",...,"19,415","18,842","18,367","18,374","18,152","18,443","17,969",-474,"-2,300","-1,434"
4,6대광역시,부산,"8,111","8,111","9,078","8,965","9,402","9,501","9,453","9,457",...,"9,279","9,327","9,345","9,515","9,559","9,581","9,608",1,430,477
5,NaN,대구,"8,080","8,080","8,077","8,101","8,267","8,274","8,360","8,360",...,"8,441","8,446","8,568","8,542","8,542","8,795","8,863",27,400,350
6,NaN,인천,"10,204","10,204","10,408","10,408","10,000","9,844","10,058","9,974",...,"9,876","9,938","10,551","10,443","10,443","10,449","10,450",-162,-150,-131
7,NaN,광주,"6,098","7,326","7,611","7,346","7,346","7,523","7,659","7,612",...,"7,861","7,914","7,877","7,881","8,089","8,231","8,083",-148,334,281
8,NaN,대전,"8,321","8,321","8,321","8,341","8,341","8,341","8,333","8,333",...,"8,067","8,145","8,272","8,079","8,079","8,079","7,917",68,610,414
9,NaN,울산,"8,090","8,090","8,090","8,153","8,153","8,153","8,153","8,153",...,"8,629","9,380","9,192","9,190","9,190","9,215","9,215",0,324,722
10,NaN,NaN,"8,151","8,355","8,597","8,552","8,585","8,606","8,669","8,648",...,"8,692","8,858","8,967","8,942","8,984","9,058","9,023",-36,325,352
11,경기,NaN,"10,855","10,855","10,791","10,784","10,876","10,646","10,266","10,124",...,"10,469","10,684","10,685","10,573","10,518","10,573","10,341",-232,-38,-160


In [188]:
df2.reset_index(drop=True)

,구분,시군구,2013년,2014년 1월,2014년 2월,2014년 3월,2014년 4월,2014년 5월,2014년 6월,2014년 7월,...,2015년 3월,2015년 4월,2015년 5월,2015년 6월,2015년 7월,2015년 8월,2015년 9월,2015년 전월비,2015년 전년말비,2015년 전년동월비
0,전국,NaN,"8,059","8,130","8,195","8,204","8,289","8,358","8,344","8,333",...,"8,563","8,613","8,624","8,627","8,643","8,678","8,665",-13,82,207
1,서울,NaN,"18,189","17,925","17,925","18,016","18,098","19,446","18,867","18,742",...,"19,415","18,842","18,367","18,374","18,152","18,443","17,969",-474,"-2,300","-1,434"
2,6대광역시,부산,"8,111","8,111","9,078","8,965","9,402","9,501","9,453","9,457",...,"9,279","9,327","9,345","9,515","9,559","9,581","9,608",1,430,477
3,NaN,대구,"8,080","8,080","8,077","8,101","8,267","8,274","8,360","8,360",...,"8,441","8,446","8,568","8,542","8,542","8,795","8,863",27,400,350
4,NaN,인천,"10,204","10,204","10,408","10,408","10,000","9,844","10,058","9,974",...,"9,876","9,938","10,551","10,443","10,443","10,449","10,450",-162,-150,-131
5,NaN,광주,"6,098","7,326","7,611","7,346","7,346","7,523","7,659","7,612",...,"7,861","7,914","7,877","7,881","8,089","8,231","8,083",-148,334,281
6,NaN,대전,"8,321","8,321","8,321","8,341","8,341","8,341","8,333","8,333",...,"8,067","8,145","8,272","8,079","8,079","8,079","7,917",68,610,414
7,NaN,울산,"8,090","8,090","8,090","8,153","8,153","8,153","8,153","8,153",...,"8,629","9,380","9,192","9,190","9,190","9,215","9,215",0,324,722
8,NaN,NaN,"8,151","8,355","8,597","8,552","8,585","8,606","8,669","8,648",...,"8,692","8,858","8,967","8,942","8,984","9,058","9,023",-36,325,352
9,경기,NaN,"10,855","10,855","10,791","10,784","10,876","10,646","10,266","10,124",...,"10,469","10,684","10,685","10,573","10,518","10,573","10,341",-232,-38,-160
